## **<span style="color:#023e8a;">Intro</span>**

**<span style="color:#023e8a;">The competition is dedicated to the product recomendations (H&M)  </span>**

**<span style="color:#023e8a;">Here we have different kinds of data that help us to get good recomendations: </span>**

📸 `images` - images of every article_id

🙋 `articles`  - detailed metadata of every article_id

👔 `customers`  - detailed metadata of every customer_id

🧾 `transactions_train`  - purchases with details

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
articles = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
customers = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")
transactions = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

## **<span id="Articles" style="color:#023e8a;">2. Articles</span>**

**<span style="color:#023e8a;"> This table contains all h&m articles with details such as a type of product, a color, a product group and other features.</span>**  
**<span style="color:#023e8a;"> Article data description: </span>**

> `article_id` **<span style="color:#023e8a;">: A unique identifier of every article.</span>**  
> `product_code`, `prod_name` **<span style="color:#023e8a;">: A unique identifier of every product and its name (not the same).</span>**  
> `product_type`, `product_type_name` **<span style="color:#023e8a;">: The group of product_code and its name</span>**  
> `graphical_appearance_no`, `graphical_appearance_name` **<span style="color:#023e8a;">: The group of graphics and its name</span>**  
> `colour_group_code`, `colour_group_name` **<span style="color:#023e8a;">: The group of color and its name</span>**  
> `graphical_appearance_no`, `graphical_appearance_name` **<span style="color:#023e8a;">: The group of graphics and its name</span>**  
> `perceived_colour_value_id`, `perceived_colour_value_name`, `perceived_colour_master_id`, `perceived_colour_master_name` **<span style="color:#023e8a;">: The added color info</span>**  
> `department_no`, `department_name`: **<span style="color:#023e8a;">: A unique identifier of every dep and its name</span>**  
> `index_code`, `index_name`: **<span style="color:#023e8a;">: A unique identifier of every index and its name</span>**  
> `index_group_no`, `index_group_name`: **<span style="color:#023e8a;">: A group of indeces and its name</span>**  
> `section_no`, `section_name`: **<span style="color:#023e8a;">: A unique identifier of every section and its name</span>**  
> `garment_group_no`, `garment_group_name`: **<span style="color:#023e8a;">: A unique identifier of every garment and its name</span>**  
> `detail_desc`: **<span style="color:#023e8a;">: Details</span>**  

In [ ]:
articles.head()

In [ ]:
len(articles)

**<span style="color:#023e8a;"> Customers data description: </span>**

> `customer_id` **<span style="color:#023e8a;">: A unique identifier of every customer</span>**  
> `FN` **<span style="color:#023e8a;">: 1 or missed </span>**  
> `Active` **<span style="color:#023e8a;">: 1 or missed</span>**  
> `club_member_status` **<span style="color:#023e8a;">: Status in club</span>**  
> `fashion_news_frequency` **<span style="color:#023e8a;">: How often H&M may send news to customer</span>**  
> `age` **<span style="color:#023e8a;">: The current age</span>**  
> `postal_code` **<span style="color:#023e8a;">: Postal code of customer</span>**  

In [ ]:
pd.options.display.max_rows = 50
customers.head()

In [ ]:
#quick_bar_chart('postal_code', (3, 3), False, customers, True)
len(customers)

In [ ]:
import sys
import time

import logging
import threading

import math

In [ ]:
class ProgressBar:
    
    def __init__(self, transactions, trans_len = None):
        self.percent_intervals = 3
        self.percent_rounded_decimals = 2
        if trans_len == None:
            self.trans_len = len(transactions)
        else:
            self.trans_len = trans_len
        self.percent = int(self.trans_len / 10**self.percent_intervals)
        self.epsilon = 10**-2
        self.start = time.time()

    def check(self, i, chunked=False):
        if i % self.percent == 0 or i + 1 == self.trans_len or chunked:
            end = time.time()
            percent_decimal = i / (self.trans_len - 1)
            percent_current = percent_decimal * 100
            time_elapsed = end - self.start
            time_estimated = time_elapsed / (percent_decimal + self.epsilon)
            sys.stdout.write("%6.2f%% time elapsed: %d, estimated: %d\r" % (percent_current, time_elapsed, time_estimated))

In [ ]:
articles_id_hash = {}
pb = ProgressBar(articles)
for i in range(len(articles)):
    columns = ['prod_name',
    'product_type_name',
    'product_group_name',
    'graphical_appearance_name',
    'colour_group_name',
    'department_name',
    'index_name',
    'index_group_name',
    'section_name',
    'garment_group_name',
    'detail_desc']
    row = []
    for column in columns:
        row.append(articles[column][i])
    articles_id = articles['article_id'][i]
    articles_id_hash[articles_id] = row
    pb.check(i)

In [ ]:
articles.head()

In [ ]:
customers_id_hash = {}
nan_count = 0
pb = ProgressBar(customers)
for i in range(len(customers)):
    customer_age = customers['age'][i]
    customer_id = customers['customer_id'][i]
    customers_id_hash[customer_id] = customer_age
    if math.isnan(customer_age):
        nan_count += 1
    pb.check(i)

In [ ]:
nan_count / len(customers) * 100

In [ ]:
customers_id_hash['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657']

In [ ]:
np.max(customers['age']), np.min(customers['age']), np.mean(customers['age']), np.median(customers['age'])

## **<span id="Transactions" style="color:#023e8a;">4. Transactions</span>**

[**<span style="color:#FEF1FE;background-color:#023e8a;border-radius: 5px;padding: 2px">Go to Table of Content</span>**](#Content)

**<span style="color:#023e8a;"> Transactions data description: </span>**

> `t_dat` **<span style="color:#023e8a;">: A unique identifier of every customer</span>**  
> `customer_id` **<span style="color:#023e8a;">: A unique identifier of every customer </span>**  **<span style="color:#FF0000;">(in </span>** `customers` **<span style="color:#FF0000;"> table)</span>**  
> `article_id` **<span style="color:#023e8a;">: A unique identifier of every article</span>**  **<span style="color:#FF0000;">(in </span>** `articles` **<span style="color:#FF0000;"> table)</span>**  
> `price` **<span style="color:#023e8a;">: Price of purchase</span>**  
> `sales_channel_id` **<span style="color:#023e8a;">: 1 or 2</span>**  

In [ ]:
transactions.head()

In [ ]:
len(transactions)

In [ ]:
percent_intervals = 3
percent_rounded_decimals = 2
trans_len = len(transactions)
percent = int(trans_len / 10**percent_intervals)
epsilon = 10**-2
start = time.time()
for i in range(0,len(transactions),30):
    transactions['t_dat'][i]
    if i % percent == 0 or i + 1 == trans_len:
        end = time.time()
        percent_decimal = i / (trans_len - 1)
        percent_current = percent_decimal * 100
        time_elapsed = end - start
        time_estimated = time_elapsed / (percent_decimal + epsilon)
        sys.stdout.write("%6.2f%% time elapsed: %d, estimated: %d\r" % (percent_current, time_elapsed, time_estimated))

In [ ]:
t_dat = transactions['t_dat']
pb = ProgressBar(transactions)
for i in range(0,len(transactions),1):
    t_dat[i]
    pb.check(i)

In [ ]:
speed_list = []
pb = ProgressBar(transactions)
for i in range(0,len(transactions),1):
    sub_list = []
    sub_list.append(transactions['t_dat'][i])
    sub_list.append(transactions['customer_id'][i])
    sub_list.append(transactions['article_id'][i])
    sub_list.append(transactions['price'][i])
    speed_list.append(sub_list)
    pb.check(i)

In [ ]:
len(speed_list)

In [ ]:
new_list = []
pb = ProgressBar(speed_list)
for i in range(0,len(speed_list)):
    new_list.append(speed_list[i][2])
    if i % 1000000 == 0:
        print(speed_list[i][2])
    pb.check(i)

In [ ]:
def thread_function(name):
    print("Thread %s: starting", name)
    time.sleep(3 - name)
    print("Thread %s: finishing", name)


In [ ]:
print("Main    : before creating thread")
for i in range(0,2):
    x = threading.Thread(target=thread_function, args=(i,))
    print("Main    : before running thread")
    x.start()
print("Main    : wait for the thread to finish")
# x.join()
print("Main    : all done")

In [ ]:
#speed_list = []
#def thread_function(name):
#    pb = 0
#    if name == 0:
#        pb = ProgressBar(transactions)
#    for i in range(0, len(transactions)):
#        if name == 0 and i % 2 == 0:
#            continue
#        elif name == 1 and i % 2 == 1:
#            continue
#        speed_list.append(transactions['t_dat'][i])
#        if name == 0:
#            pb.check(i)
#            
#    print("done",name)
#        
#for i in range(0,2):
#    x = threading.Thread(target=thread_function, args=(i,))
#    print("Main    : before running thread")
#    x.start()

In [ ]:
customers.head()

In [ ]:
transactions.head()

In [ ]:
transactions_age_list = []
pb = ProgressBar(transactions)
for i in range(0,len(transactions),100):
    customer_id = transactions['customer_id'][i]
    transactions_age_list.append(customers_id_hash[customer_id])
    pb.check(i, True)

In [ ]:
plt.figure(figsize=(10, 6), dpi=80)
n, bins, patches = plt.hist(transactions_age_list, 99-16, density=False, facecolor='#CC071E')#, facecolor='g', alpha=0.75)
plt.xlabel('Age (Years)')
plt.ylabel('Count')
plt.title('Histogram of H&M Customer Ages Transaction')
plt.xlim(15, 100)
plt.ylim(0, 20000)
plt.grid(True)
plt.xticks(np.arange(15, 105, step=5))
plt.show()

In [ ]:
age_article_id_list = {}
pb = ProgressBar(transactions)
for i in range(0,len(transactions),1):
    customer_id = transactions['customer_id'][i]
    article_id = transactions['article_id'][i]
    customer_age = customers_id_hash[customer_id]
    if math.isnan(customer_age):
        customer_age = 'nan'
    else:
        customer_age = str(int(customer_age))
    if not customer_age in age_article_id_list:
        age_article_id_list[customer_age] = {}
    if not article_id in age_article_id_list[customer_age]:
        age_article_id_list[customer_age][article_id] = 0
    age_article_id_list[customer_age][article_id] += 1
    
    pb.check(i, True)

In [ ]:
speed_list = []
pb = ProgressBar(transactions)
for i in range(0,len(transactions),100):
    sub_list = []
    sub_list.append(transactions['t_dat'][i])
    sub_list.append(transactions['customer_id'][i])
    sub_list.append(transactions['article_id'][i])
    sub_list.append(transactions['price'][i])
    speed_list.append(sub_list)
    pb.check(i)

In [ ]:
age_article_id_list = {}
pb = ProgressBar(speed_list)
for i in range(0,len(speed_list),1):
    customer_id = speed_list[i][1]
    article_id = speed_list[i][2]
    customer_age = customers_id_hash[customer_id]
    if math.isnan(customer_age):
        customer_age = 'nan'
    else:
        customer_age = str(int(customer_age))
    if not customer_age in age_article_id_list:
        age_article_id_list[customer_age] = {}
    if not article_id in age_article_id_list[customer_age]:
        age_article_id_list[customer_age][article_id] = 0
    age_article_id_list[customer_age][article_id] += 1
    
    pb.check(i, True)

In [ ]:
age_article_id_list.keys()

In [ ]:
def sort_obj_by_keys(obj, sort_on_keys=True, reverse=False):
    
    keys = []
    values = []
    for key in obj.keys():
        keys.append(key)
        values.append(obj[key])
    
    ind = None
    if sort_on_keys:
        ind = np.argsort(keys)
    else:
        ind = np.argsort(values)
    
    keys_sort = np.array(keys)[ind]
    values_sort = np.array(values)[ind]
    
    if reverse:
        keys_sort = np.flip(keys_sort)
        values_sort = np.flip(values_sort)
    
    newObj = {}
    newObj['keys'] = keys_sort
    newObj['values'] = values_sort
    
    return newObj

In [ ]:
age_article_id_list_sort = sort_obj_by_keys(age_article_id_list)

In [ ]:
#age_article_id_list_sort

In [ ]:
values = age_article_id_list_sort['values']
newValues = []
for i in range(len(values)):
    value = values[i]
    newValue = sort_obj_by_keys(value, False, True)
    newValues.append(newValue)

In [ ]:
age_article_id_list_sort['values'] = newValues

In [ ]:
for i in range(len(age_article_id_list_sort['keys'])):
    age = age_article_id_list_sort['keys'][i]
    articles_per_age = age_article_id_list_sort['values'][i]
    
    print('Age:',age)
    end = 10
    if len(articles_per_age['keys']) < 10:
        end = len(articles_per_age['keys'])
    for j in range(0,end):
        article_id = articles_per_age['keys'][j]
        article_count = articles_per_age['values'][j]
        
        print('Count:',article_count, articles_id_hash[article_id][0])#, articles_id_hash[article_id][10])
        
    print()

In [ ]:
transactions['article_id'][0]

In [ ]:
transactions['article_id']

In [ ]:
age_article_id_list = {}
pb = ProgressBar(transactions)
for i in range(0,len(transactions),100):
    customer_id = transactions['customer_id'][i]
    article_id = transactions['article_id'][i]
    customer_age = customers_id_hash[customer_id]
    if math.isnan(customer_age):
        customer_age = 'nan'
    else:
        customer_age = str(int(customer_age/5))
    if not customer_age in age_article_id_list:
        age_article_id_list[customer_age] = {}
    if not article_id in age_article_id_list[customer_age]:
        age_article_id_list[customer_age][article_id] = 0
    age_article_id_list[customer_age][article_id] += 1
    
    pb.check(i, True)

In [ ]:
age_article_id_list_sort = sort_obj_by_keys(age_article_id_list)

In [ ]:
values = age_article_id_list_sort['values']
newValues = []
for i in range(len(values)):
    value = values[i]
    newValue = sort_obj_by_keys(value, False, True)
    newValues.append(newValue)

In [ ]:
age_article_id_list_sort['values'] = newValues

In [ ]:
for i in range(len(age_article_id_list_sort['keys'])):
    age = age_article_id_list_sort['keys'][i]
    articles_per_age = age_article_id_list_sort['values'][i]
    
    print('Age:',age)
    end = 10
    if len(articles_per_age['keys']) < 10:
        end = len(articles_per_age['keys'])
    for j in range(0,end):
        article_id = articles_per_age['keys'][j]
        article_count = articles_per_age['values'][j]
        
        print('Count:',article_count, articles_id_hash[article_id][0])#, articles_id_hash[article_id][10])
        
    print()

In [ ]:
import csv

# open the file in the write mode
f = open('submission.csv', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(['customer_id','prediction'])

pb = ProgressBar(customers)
for i in range(len(customers['customer_id'])):
    customer_id = customers['customer_id'][i]
    customer_age = customers['age'][i]
    article_id = articles['article_id'][0]
    if math.isnan(customer_age):
        customer_age = 'nan'
    else:
        customer_age = str(int(customer_age/5))
    
    article_ids = ''
    for j in range(0,len(age_article_id_list_sort['keys'])):
        key_age = age_article_id_list_sort['keys'][j]
        if key_age == customer_age:
            values_age = age_article_id_list_sort['values'][j]
            end = 10
            if len(values_age['keys']) < 10:
                end = len(values_age['keys'])
            for k in range(0,end):
                article_id = str(values_age['keys'][k])
                for l in range(len(article_id), 10):
                    article_id = '0' + article_id
                article_ids += str(article_id) + ' '
    writer.writerow([customer_id, article_ids])
    pb.check(i)
# close the file
f.close()

In [ ]:
#age_article_id_list_sort

In [ ]:
len(age_article_id_list_sort['values'][0]['keys'][0])